<a href="https://colab.research.google.com/github/HOowada/WorkshopArchive/blob/master/EDINET_INFO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SET_UP

In [ ]:
# !nvidia-smi

In [ ]:
!pip install --upgrade pip

# EDINET API
!pip install edinet-xbrl

# Arelle
!pip install arelle
!pip install lxml
!pip install isodate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.2 MB/s eta 0:00:00
  Created wheel for edinet-xbrl: filename=edinet_xbrl-0.2.0-py2.py3-none-any.whl size=10849 sha256=e6ac972afd0708f5706c07711594646f6afa294fa6f30b96cb78891e963310c6
  Stored in directory: /root/.cache/pip/wheels/9f/e5/02/7a2fb1d397b2e5e9badd3ce824287e8469404491af0d346de3
  Created wheel for python-xbrl: filename=python_xbrl-1.1.1-py2.py3-none-any.whl size=10613 sha256=0c22035a41c9433eb6a89319db87349c4c8f9e1fc8acf221181d749d568920f9
  Stored in directory: /root/.cache/pip/wheels/6e/24/e6/16c20c0ef

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# --------------------
# 各種ライブラリーのインポート
# --------------------
import os
import warnings
warnings.simplefilter('ignore')
import gc

import requests
from pprint import pprint
import glob
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 150)
# from google.colab.data_table import DataTable
# DataTable.max_columns = 300

from tqdm.notebook import tqdm
import datetime
import time
import datetime as dt
import zipfile

In [ ]:
start_date = datetime.date(2023, 6, 1)
end_date = datetime.date(2023, 6,30)

start_date = datetime.date(2023, 1, 1)
end_date = datetime.date(2023, 12,31)

# EDINET_API

In [ ]:
# -----------------------------
# EDINET API 仕様書（Version 1）
# -----------------------------
# 3-1 参考資料: 書類種別コード
#
docTypeCodeList = {'010':'有価証券通知書', '020':'変更通知書（有価証券通知書）', '030':'有価証券届出書',
                   '040':'訂正有価証券届出書', '050':'届出の取下げ願い', '060':'発行登録通知書',
                   '070':'変更通知書（発行登録通知書）', '080':'発行登録書', '090':'訂正発行登録書',
                   '100':'発行登録追補書類', '110':'発行登録取下届出書', '120':'有価証券報告書',
                   '130':'訂正有価証券報告書', '135':'確認書', '136':'訂正確認書', '140':'四半期報告書',
                   '150':'訂正四半期報告書', '160':'半期報告書', '170':'訂正半期報告','180':'臨時報告書',
                   '190':'訂正臨時報告書', '200':'親会社等状況報告書', '210':'訂正親会社等状況報告書',
                   '220':'自己株券買付状況報告書', '230':'訂正自己株券買付状況報告書', '235':'内部統制報告書',
                   '236':'訂正内部統制報告書', '240':'公開買付届出書', '250':'訂正公開買付届出書',
                   '260':'公開買付撤回届出書', '270':'公開買付報告書', '280':'訂正公開買付報告書',
                   '290':'意見表明報告書', '300':'訂正意見表明報告書', '310':'対質問回答報告書',
                   '320':'訂正対質問回答報告書', '330':'別途買付け禁止の特例を受けるための申出書',
                   '340':'訂正別途買付け禁止の特例を受けるための申出書', '350':'大量保有報告書',
                   '360':'訂正大量保有報告書', '370':'基準日の届出書', '380':'変更の届出書',}

In [ ]:
period = end_date - start_date
period = int(period.days)
day_list = []
for d in range(period):
    day = start_date + datetime.timedelta(days=d)
    day_list.append(day)

day_list.append(end_date)

In [ ]:
#結果を格納するための空のリストを用意します
report_list =[]
#日付リストの期間に提出された書類のメタデータを取得してjson形式に変換します
for day in day_list:
    url = "https://disclosure.edinet-fsa.go.jp/api/v1/documents.json"
    params = {"date": day, "type": 2}
    res = requests.get(url, params=params)
    try:
        json_data = res.json()
    except Exception:
        pass

    for num in range(len(json_data["results"])):
        ordinance_code= json_data["results"][num]["ordinanceCode"]
        form_code= json_data["results"][num]["formCode"]

    #ordinance_code=010かつform_code=030000が有価証券報告書になります
        if ordinance_code == "010" and  form_code =="030000" :
            company_name=json_data["results"][num]["filerName"]
            edi={ '会社名':company_name,
                        '書類名':json_data["results"][num]["docDescription"],
                        'docID':json_data["results"][num]["docID"],
                        '証券コード':json_data["results"][num]["secCode"],
                        '日付': day             }
            report_list.append(edi)

NameError: name 'json_data' is not defined

In [ ]:
df = pd.DataFrame(report_list)

# エラーが出るものを除き書類管理番号リストを作成
docid_list=df["docID"].to_list()

In [ ]:
df.to_csv("/content/drive/MyDrive/study/NLP_all/export/company_list_FY2023_3")

OSError: Cannot save file into a non-existent directory: '/content/drive/MyDrive/study/NLP_all/export'

### ファイルの読み込み

In [ ]:
filename_list = []

for docid in tqdm(docid_list):
    # 書類取得APIのエンドポイント
    url = "https://disclosure.edinet-fsa.go.jp/api/v1/documents/" + docid
    print(url)

    # 書類取得APIのリクエストパラメータ
    params = {
        "type" : 1}

    # 書類取得APIの呼び出し
    res = requests.get(url, params=params, verify=False)
    # ファイルへ出力
    # print(res.status_code)
    if res.status_code == 200:
      filename_list.append(docid + ".zip")
      with open("/content/"+docid+".zip", 'wb') as f:
        for chunk in res.iter_content(chunk_size=1024):
          f.write(chunk)

  0%|          | 0/2571 [00:00<?, ?it/s]

https://disclosure.edinet-fsa.go.jp/api/v1/documents/S100QVA8
https://disclosure.edinet-fsa.go.jp/api/v1/documents/S100QVU7
https://disclosure.edinet-fsa.go.jp/api/v1/documents/S100QWA7
https://disclosure.edinet-fsa.go.jp/api/v1/documents/S100QWJX
https://disclosure.edinet-fsa.go.jp/api/v1/documents/S100QWIX
https://disclosure.edinet-fsa.go.jp/api/v1/documents/S100QWQB
https://disclosure.edinet-fsa.go.jp/api/v1/documents/S100QWBV
https://disclosure.edinet-fsa.go.jp/api/v1/documents/S100QWNJ
https://disclosure.edinet-fsa.go.jp/api/v1/documents/S100QX0Q
https://disclosure.edinet-fsa.go.jp/api/v1/documents/S100QXL5
https://disclosure.edinet-fsa.go.jp/api/v1/documents/S100QXDB
https://disclosure.edinet-fsa.go.jp/api/v1/documents/S100QXCJ
https://disclosure.edinet-fsa.go.jp/api/v1/documents/S100QXNZ
https://disclosure.edinet-fsa.go.jp/api/v1/documents/S100QXM7
https://disclosure.edinet-fsa.go.jp/api/v1/documents/S100QXOD
https://disclosure.edinet-fsa.go.jp/api/v1/documents/S100QX95
https://

KeyboardInterrupt: 

In [ ]:
filename_list

['S100QVA8.zip',
 'S100QVU7.zip',
 'S100QWA7.zip',
 'S100QWJX.zip',
 'S100QWIX.zip',
 'S100QWQB.zip',
 'S100QWBV.zip',
 'S100QWNJ.zip',
 'S100QX0Q.zip',
 'S100QXL5.zip',
 'S100QXDB.zip',
 'S100QXCJ.zip',
 'S100QXNZ.zip',
 'S100QXM7.zip',
 'S100QXOD.zip',
 'S100QX95.zip',
 'S100QXSH.zip',
 'S100QY33.zip',
 'S100QXQN.zip',
 'S100QY5Z.zip',
 'S100QXGZ.zip',
 'S100QY25.zip',
 'S100QY5R.zip',
 'S100QXYU.zip',
 'S100QYD3.zip',
 'S100QY5M.zip',
 'S100QY43.zip',
 'S100QY7Y.zip',
 'S100QYG8.zip',
 'S100QY8D.zip',
 'S100QY64.zip',
 'S100QY1D.zip',
 'S100QXXT.zip',
 'S100QXRZ.zip',
 'S100QYE0.zip',
 'S100QYKD.zip',
 'S100QYIH.zip',
 'S100QY1X.zip',
 'S100QY6Q.zip',
 'S100QYFK.zip',
 'S100QYOR.zip',
 'S100QYPF.zip',
 'S100QY9X.zip',
 'S100QYPN.zip',
 'S100QYNB.zip',
 'S100QYQ4.zip',
 'S100QYF2.zip',
 'S100QXYT.zip',
 'S100QYFA.zip',
 'S100QY82.zip',
 'S100QYHG.zip',
 'S100QX4W.zip',
 'S100QX7W.zip',
 'S100QYSG.zip',
 'S100QY6D.zip',
 'S100QXO0.zip',
 'S100QYHE.zip',
 'S100QYHC.zip',
 'S100QYLX.zip

In [ ]:
for i, docid in enumerate(docid_list):
    # print(filename_list[i])
    if zipfile.is_zipfile(filename_list[i]) == True:
      with zipfile.ZipFile(filename_list[i]) as zip_f:
        zip_f.extractall(docid)
    if zipfile.is_zipfile(filename_list[i]) == False: #エラー回避
      print(filename_list[i] + " is not a Zip File")
      docid_list.remove(docid)

KeyboardInterrupt: ignored

In [ ]:
# for i, docid in enumerate(docid_list):
#     is_file = os.path.isfile(filename_list[i])
#     if is_file:
#         with zipfile.ZipFile(filename_list[i]) as zip_f:
#             zip_f.extractall(docid)
#     else:
#         pass

IndexError: ignored

## ファイルからテキストデータ取得

In [ ]:
from edinet_xbrl.edinet_xbrl_parser import EdinetXbrlParser
#インスタンス化
parser = EdinetXbrlParser()

ModuleNotFoundError: ignored

In [ ]:
docid

NameError: ignored

In [ ]:
#パーサのインスタンス化
parser = EdinetXbrlParser()
docid = "S100QXHM"

#xbrlファイルのパスを指定
xbrl_path = './'+ docid + '/XBRL/PublicDoc/*.xbrl'
xbrl_path = glob.glob(xbrl_path)[0]

edinet_xbrl_object = parser.parse_file(xbrl_path)

#①事業等のリスクをとってみる
key='jpcrp_cor:GovernanceTextBlock'
context_ref='FilingDateInstant'
context_ref_2='CurrentYearDuration'
data = edinet_xbrl_object.get_data_by_context_ref(key, context_ref)
text_data = data.get_value()
print(text_data)

# #②経営方針、経営環境および対処すべき課題等をとってみる
# key='jpcrp_cor:BusinessPolicyBusinessEnvironmentIssuesToAddressEtcTextBlock'
# context_ref='FilingDateInstant'
# data = edinet_xbrl_object.get_data_by_context_ref(key, context_ref)
# text_data = data.get_value()

NameError: ignored

### 参照


In [ ]:
docid_list

['S100QVA8',
 'S100QVU7',
 'S100QWA7',
 'S100QWJX',
 'S100QWIX',
 'S100QWQB',
 'S100QWBV',
 'S100QWNJ',
 'S100QX0Q',
 'S100QXL5',
 'S100QXDB',
 'S100QXCJ',
 'S100QXNZ',
 'S100QXM7',
 'S100QXOD',
 'S100QX95',
 'S100QXSH',
 'S100QY33',
 'S100QXQN',
 'S100QY5Z',
 'S100QXGZ',
 'S100QY25',
 'S100QY5R',
 'S100QXYU',
 'S100QYD3',
 'S100QY5M',
 'S100QY43',
 'S100QY7Y',
 'S100QYG8',
 'S100QY8D',
 'S100QY64',
 'S100QY1D',
 'S100QXXT',
 'S100QXRZ',
 'S100QYE0',
 'S100QYKD',
 'S100QYIH',
 'S100QY1X',
 'S100QY6Q',
 'S100QYFK',
 'S100QYOR',
 'S100QYPF',
 'S100QY9X',
 'S100QYPN',
 'S100QYNB',
 'S100QYQ4',
 'S100QYF2',
 'S100QXYT',
 'S100QYFA',
 'S100QY82',
 'S100QYHG',
 'S100QX4W',
 'S100QX7W',
 'S100QYSG',
 'S100QY6D',
 'S100QXO0',
 'S100QYHE',
 'S100QYHC',
 'S100QYLX',
 'S100QYVK',
 'S100QYU0',
 'S100QYSB',
 'S100QYK6',
 'S100QYK5',
 'S100QYM7',
 'S100QY0V',
 'S100QXEE',
 'S100QW7L',
 'S100QZ0Y',
 'S100QYR4',
 'S100QYOP',
 'S100QY7S',
 'S100QYB8',
 'S100QYPV',
 'S100QYXU',
 'S100QZ45',
 'S100QZ41',

In [ ]:
df = pd.DataFrame(columns=['docid',
                           '経営方針',
                          #  'サステナビリティ',
                           '事業等のリスク'])
for _docid in tqdm(docid_list):
    is_dir = os.path.isdir('/content/' + _docid)
    if is_dir: #ディレクトリが存在する場合
        #xbrlファイルのパスを指定
        xbrl_path = '/content/' + _docid + '/XBRL/PublicDoc/*.xbrl'
        print(xbrl_path)

        xbrl_path = glob.glob(xbrl_path)[0]
        print(xbrl_path)
        edinet_xbrl_object = parser.parse_file(xbrl_path)

        #keyとcontext_refを指定。keyが経営方針と事業等のリスクでは異なる
        #経営方針、経営環境、サステナビリティおよび対処すべき課題等
        key_houshin= 'jpcrp_cor:BusinessPolicyBusinessEnvironmentIssuesToAddressEtcTextBlock'

        # #サステナビリティ
        key_sus = 'jpcrp_cor:NetSalesSummaryOfBusinessResults'

        #事業等のリスク
        key_risk ='jpcrp_cor:BusinessRisksTextBlock'

        #context_refは経営方針と事業等のリスクともに共通
        context_ref='FilingDateInstant'

        #例外処理
        try:
            data_houshin = edinet_xbrl_object.get_data_by_context_ref(key_houshin, context_ref).get_value()
            soup_houshin = BeautifulSoup(data_houshin,'html.parser').get_text(strip=True)


            data_sus = edinet_xbrl_object.get_data_by_context_ref(key_sus, context_ref).get_value()
            soup_sus = BeautifulSoup(data_sus,'html.parser').get_text(strip=True)

            print(soup_sus)

            data_risk = edinet_xbrl_object.get_data_by_context_ref(key_risk, context_ref).get_value()
            soup_risk = BeautifulSoup(data_risk,'html.parser').get_text(strip=True)

            tmp_se = pd.DataFrame([[_docid], [soup_houshin],[soup_risk]], index=df.columns).T
            df = df.append(tmp_se)

        except AttributeError as e:
            print(e)
    else:
        pass

  0%|          | 0/2571 [00:00<?, ?it/s]

/content/S100QVA8/XBRL/PublicDoc/*.xbrl
/content/S100QVA8/XBRL/PublicDoc/jpcrp030000-asr-001_E02425-000_2023-03-31_01_2023-06-01.xbrl
'NoneType' object has no attribute 'get_value'
/content/S100QVU7/XBRL/PublicDoc/*.xbrl
/content/S100QVU7/XBRL/PublicDoc/jpcrp030000-asr-001_E04708-000_2023-03-20_01_2023-06-06.xbrl
'NoneType' object has no attribute 'get_value'
/content/S100QWA7/XBRL/PublicDoc/*.xbrl
/content/S100QWA7/XBRL/PublicDoc/jpcrp030000-asr-001_E01124-000_2023-03-31_01_2023-06-06.xbrl
'NoneType' object has no attribute 'get_value'
/content/S100QWJX/XBRL/PublicDoc/*.xbrl
/content/S100QWJX/XBRL/PublicDoc/jpcrp030000-asr-001_E03926-000_2023-03-31_01_2023-06-08.xbrl
'NoneType' object has no attribute 'get_value'
/content/S100QWIX/XBRL/PublicDoc/*.xbrl
/content/S100QWIX/XBRL/PublicDoc/jpcrp030000-asr-001_E04612-000_2023-03-31_01_2023-06-08.xbrl
'NoneType' object has no attribute 'get_value'
/content/S100QWQB/XBRL/PublicDoc/*.xbrl
/content/S100QWQB/XBRL/PublicDoc/jpcrp030000-asr-001_E0

KeyboardInterrupt: ignored

In [ ]:
df

,docid,経営方針,サステナビリティ,事業等のリスク


In [ ]:
df.to_csv("/content/drive/MyDrive/study/NLP_all/export/2023_sample.csv")

## 読み込みから

In [ ]:
df = pd.read_csv("../export/2022_sample.csv")

In [ ]:
df

In [ ]:
import pandas as pd
import MeCab
import subprocess
import re
from sklearn.feature_extraction.text import CountVectorizer
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel
from tqdm import tqdm
import numpy as np

In [ ]:
#stopwordsの指定
!wget http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt

with open("Japanese.txt","r") as f:
    stopwords = f.read().split("\n")

#Neologdによるトーカナイザー(リストで返す関数・名詞のみ)
def mecab_tokenizer(text):

    replaced_text = text.lower()
    replaced_text = re.sub(r'[【】]', ' ', replaced_text)       # 【】の除去
    replaced_text = re.sub(r'[（）()]', ' ', replaced_text)     # （）の除去
    replaced_text = re.sub(r'[［］\[\]]', ' ', replaced_text)   # ［］の除去
    replaced_text = re.sub(r'[@＠]\w+', '', replaced_text)  # メンションの除去
    replaced_text = re.sub(r'\d+\.*\d*', '', replaced_text) #数字を0にする


    path = "mecab-ipadic-neologd/"
    mecab = MeCab.Tagger(path)
    parsed_lines = mecab.parse(replaced_text).split("\n")[:-2]

#     cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
#     path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
#                               shell=True).communicate()[0]).decode('utf-8')

#     mecab = MeCab.Tagger("-d {0}".format(path))
#     parsed_lines = mecab.parse(replaced_text).split("\n")[:-2]

    # #表層形を取得
    # surfaces = [l.split('\t')[0] for l in parsed_lines]
    #原形を取得
    token_list = [l.split("\t")[1].split(",")[6] for l in parsed_lines]
    #品詞を取得
    pos = [l.split('\t')[1].split(",")[0] for l in parsed_lines]
    # 名詞,動詞,形容詞のみに絞り込み
    target_pos = ["名詞"]
    token_list = [t for t, p in zip(token_list, pos) if p in target_pos]

    # stopwordsの除去
    token_list = [t for t in token_list if t  not in stopwords]

    # ひらがなのみの単語を除く
    kana_re = re.compile("^[ぁ-ゖ]+$")
    token_list = [t for t in token_list if not kana_re.match(t)]

    return token_list

 #df全体に対してmecab_tokenizerを適用し、形態素解析を行なったリストを返す関数
def make_docs(df,column_number):
    docs=[]
    for i in range(len(df)):
        text = df.iloc[i,column_number]
        docs.append(mecab_tokenizer(text))
    return docs

#形態素解析の実行
docs_keiei_2203_lda = make_docs(df,2)

In [ ]:
#stopwordsの指定
# !wget http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt

with open("/content/Japanese.txt","r") as f:
    stopwords = f.read().split("\n")

#Neologdによるトーカナイザー(リストで返す関数・名詞のみ)
def mecab_tokenizer(text):

    replaced_text = text.lower()
    replaced_text = re.sub(r'[【】]', ' ', replaced_text)       # 【】の除去
    replaced_text = re.sub(r'[（）()]', ' ', replaced_text)     # （）の除去
    replaced_text = re.sub(r'[［］\[\]]', ' ', replaced_text)   # ［］の除去
    replaced_text = re.sub(r'[@＠]\w+', '', replaced_text)  # メンションの除去
    replaced_text = re.sub(r'\d+\.*\d*', '', replaced_text) #数字を0にする

    path = "-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
    mecab = MeCab.Tagger(path)
    parsed_lines = mecab.parse(replaced_text).split("\n")[:-2]

    # #表層形を取得
    # surfaces = [l.split('\t')[0] for l in parsed_lines]
    #原形を取得
    token_list = [l.split("\t")[1].split(",")[6] for l in parsed_lines]
    #品詞を取得
    pos = [l.split('\t')[1].split(",")[0] for l in parsed_lines]
    # 名詞,動詞,形容詞のみに絞り込み
    target_pos = ["名詞"]
    token_list = [t for t, p in zip(token_list, pos) if p in target_pos]

    # stopwordsの除去
    token_list = [t for t in token_list if t  not in stopwords]

    # ひらがなのみの単語を除く
    kana_re = re.compile("^[ぁ-ゖ]+$")
    token_list = [t for t in token_list if not kana_re.match(t)]

    return token_list

 #df全体に対してmecab_tokenizerを適用し、形態素解析を行なったリストを返す関数
def make_docs(df,column_number):
    docs=[]
    for i in range(len(df)):
        text = df.iloc[i,column_number]
        docs.append(mecab_tokenizer(text))
    return docs

#形態素解析の実行
docs_keiei_2203_lda = make_docs(df,2)

In [ ]:
#辞書の作成
dictionary = Dictionary(docs_keiei_2203_lda)
#出現がx文書に満たない単語と、y%以上の文書に出現する単語を極端と見做し削除する
x =500
y =0.5
dictionary.filter_extremes(no_below=x,no_above=y)
# LdaModelが読み込めるBoW形式に変換
corpus = [dictionary.doc2bow(text) for text in docs_keiei_2203_lda]

print(f"Number of unique tokens: {len(dictionary)}")
print(f"Number of documents: {len(corpus)}")

In [ ]:
num_topics =8
lda = LdaModel(corpus, id2word =dictionary, num_topics=num_topics, alpha=0.01)

In [ ]:
df =pd.DataFrame()
for t in range(num_topics):
    word=[]
    for i, prob in lda.get_topic_terms(t, topn=15):
        word.append(dictionary.id2token[int(i)])
    _ = pd.DataFrame([word],index=[f'topic{t+1}'])
    df = df.append(_)
df.T

In [ ]:
#ライブラリーのインストール
!pip install pyLDAvis

#ライブラリーのインポート
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

#PyLDAvisの実装
pyLDAvis.enable_notebook()

vis = gensimvis.prepare(
      lda, corpus, dictionary, n_jobs = 1, sort_topics = False
      )

pyLDAvis.display(vis)

# EDINET_API（PDF）

In [ ]:
# -----------------------------
# EDINET API 仕様書（Version 1）
# -----------------------------
# 3-1 参考資料: 書類種別コード
#
docTypeCodeList = {'010':'有価証券通知書', '020':'変更通知書（有価証券通知書）', '030':'有価証券届出書',
                   '040':'訂正有価証券届出書', '050':'届出の取下げ願い', '060':'発行登録通知書',
                   '070':'変更通知書（発行登録通知書）', '080':'発行登録書', '090':'訂正発行登録書',
                   '100':'発行登録追補書類', '110':'発行登録取下届出書', '120':'有価証券報告書',
                   '130':'訂正有価証券報告書', '135':'確認書', '136':'訂正確認書', '140':'四半期報告書',
                   '150':'訂正四半期報告書', '160':'半期報告書', '170':'訂正半期報告','180':'臨時報告書',
                   '190':'訂正臨時報告書', '200':'親会社等状況報告書', '210':'訂正親会社等状況報告書',
                   '220':'自己株券買付状況報告書', '230':'訂正自己株券買付状況報告書', '235':'内部統制報告書',
                   '236':'訂正内部統制報告書', '240':'公開買付届出書', '250':'訂正公開買付届出書',
                   '260':'公開買付撤回届出書', '270':'公開買付報告書', '280':'訂正公開買付報告書',
                   '290':'意見表明報告書', '300':'訂正意見表明報告書', '310':'対質問回答報告書',
                   '320':'訂正対質問回答報告書', '330':'別途買付け禁止の特例を受けるための申出書',
                   '340':'訂正別途買付け禁止の特例を受けるための申出書', '350':'大量保有報告書',
                   '360':'訂正大量保有報告書', '370':'基準日の届出書', '380':'変更の届出書',}

In [ ]:
period = end_date - start_date
period = int(period.days)
day_list = []
for d in range(period):
    day = start_date + datetime.timedelta(days=d)
    day_list.append(day)

day_list.append(end_date)

In [ ]:
#結果を格納するための空のリストを用意します
report_list =[]
#日付リストの期間に提出された書類のメタデータを取得してjson形式に変換します
for day in day_list:
    url = "https://disclosure.edinet-fsa.go.jp/api/v1/documents.json"
    params = {"date": day, "type": 2}
    res = requests.get(url, params=params)
    print(res)
    try:
        json_data = res.json()
    except Exception:
        pass

    for num in range(len(json_data["results"])):
        ordinance_code= json_data["results"][num]["ordinanceCode"]
        form_code= json_data["results"][num]["formCode"]

    #ordinance_code=010かつform_code=030000が有価証券報告書になります
        if ordinance_code == "010" and  form_code =="030000" :
            company_name=json_data["results"][num]["filerName"]
            edi={ '会社名':company_name,
                        '書類名':json_data["results"][num]["docDescription"],
                        'docID':json_data["results"][num]["docID"],
                        '証券コード':json_data["results"][num]["secCode"],
                        '日付': day             }
            report_list.append(edi)

<Response [403]>


NameError: name 'json_data' is not defined

In [ ]:
df = pd.DataFrame(report_list)

In [ ]:
df.shape

(2592, 5)

In [ ]:
df.dropna(subset=["証券コード"]).shape

(2440, 5)

In [ ]:
df[df["証券コード"]==34070]

,会社名,書類名,docID,証券コード,日付


In [ ]:


# エラーが出るものを除き書類管理番号リストを作成
docid_list=df["docID"].to_list()

In [ ]:
df

,会社名,書類名,docID,証券コード,日付
0,株式会社きもと,有価証券報告書－第63期(2022/04/01－2023/03/31),S100QVA8,79080,2023-06-01
1,株式会社京ヶ野ゴルフ倶楽部,有価証券報告書－第37期(2022/03/21－2023/03/20),S100QVU7,None,2023-06-06
2,ＨＯＹＡ株式会社,有価証券報告書－第85期(2022/04/01－2023/03/31),S100QWA7,77410,2023-06-06
3,鷹之台ゴルフ株式会社,有価証券報告書－第55期(2022/04/01－2023/03/31),S100QWJX,None,2023-06-08
4,福井県観光開発株式会社,有価証券報告書－第64期(2022/04/01－2023/03/31),S100QWIX,None,2023-06-08
...,...,...,...,...,...
2566,東洋建設株式会社,有価証券報告書－第103期(2022/04/01－2023/03/31),S100RB2H,18900,2023-06-30
2567,株式会社ヤシマキザイ,有価証券報告書－第79期(2022/04/01－2023/03/31),S100RB26,76770,2023-06-30
2568,ＲＩＺＡＰグループ株式会社,有価証券報告書－第20期(2022/04/01－2023/03/31),S100RB27,29280,2023-06-30
2569,株式会社エムケイシステム,有価証券報告書－第35期(2022/04/01－2023/03/31),S100R9Q3,39100,2023-06-30


In [ ]:
df.to_csv("/content/drive/MyDrive/study/NLP_all/export/company_list_FY2023_3")

OSError: Cannot save file into a non-existent directory: '/content/drive/MyDrive/study/NLP_all/export'

### ファイルの読み込み

In [ ]:
# docid_list

In [ ]:
filename_list = []

for docid in tqdm(docid_list):
    # 書類取得APIのエンドポイント
    url = "https://disclosure.edinet-fsa.go.jp/api/v1/documents/" + docid
    print(url)

    # 書類取得APIのリクエストパラメータ
    params = {
      "type" : 2
    }
    # /content/drive/MyDrive/有価証券報告書_pdf出力
    d = df[df["docID"]==docid]
    company_code = d["証券コード"].values[0]
    company_name = d["会社名"].values[0]
    # 出力ファイル名
    filename = f"/content/drive/MyDrive/有価証券報告書_pdf出力/{company_code}_{company_name}_{docid}.pdf"
    print(filename)
    # 書類取得APIの呼び出し
    res = requests.get(url, params=params, verify=False)

    # ファイルへ出力_
    if res.status_code == 200:
      with open(filename, 'wb') as f:
        for chunk in res.iter_content(chunk_size=1024):
          f.write(chunk)

  0%|          | 0/2571 [00:00<?, ?it/s]

ストリーミング出力は最後の 5000 行に切り捨てられました。
https://disclosure.edinet-fsa.go.jp/api/v1/documents/S100QY7S
/content/drive/MyDrive/有価証券報告書_pdf出力/17380_株式会社ニットー_S100QY7S.pdf
https://disclosure.edinet-fsa.go.jp/api/v1/documents/S100QYB8
/content/drive/MyDrive/有価証券報告書_pdf出力/15180_三井松島ホールディングス株式会社_S100QYB8.pdf
https://disclosure.edinet-fsa.go.jp/api/v1/documents/S100QYPV
/content/drive/MyDrive/有価証券報告書_pdf出力/46740_株式会社クレスコ_S100QYPV.pdf
https://disclosure.edinet-fsa.go.jp/api/v1/documents/S100QYXU
/content/drive/MyDrive/有価証券報告書_pdf出力/42990_株式会社ハイマックス_S100QYXU.pdf
https://disclosure.edinet-fsa.go.jp/api/v1/documents/S100QZ45
/content/drive/MyDrive/有価証券報告書_pdf出力/94280_株式会社クロップス_S100QZ45.pdf
https://disclosure.edinet-fsa.go.jp/api/v1/documents/S100QZ41
/content/drive/MyDrive/有価証券報告書_pdf出力/58310_株式会社しずおかフィナンシャルグループ_S100QZ41.pdf
https://disclosure.edinet-fsa.go.jp/api/v1/documents/S100QYYO
/content/drive/MyDrive/有価証券報告書_pdf出力/46260_太陽ホールディングス株式会社_S100QYYO.pdf
https://disclosure.edinet-fsa.go.jp/api/v1/document